# Import libraries

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

import json

import re

from tqdm import tqdm

from itertools import combinations

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)

In [3]:
path_to_save = './../results/03a_features'

# Loading data

In [4]:
path_to_ftir_data_3800_2500 = './../data/processed_data/dataset_3800_2500_extracted.csv'
path_to_ftir_data_1900_800= './../data/processed_data/dataset_1900_800_extracted.csv'

## FTIR-features (3800-2500 cm$^{-1}$)

In [5]:
dataset_3800_2500 = pd.read_csv(path_to_ftir_data_3800_2500, sep=';', index_col=0)

print(dataset_3800_2500.shape)
dataset_3800_2500.head(3)

(75, 12)


,$I_{2928}$,$I_{3355}$,$I_{3725}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3355 \pm 50}$,$\overline{I}_{3725 \pm 50}$,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,0.491720,1.0,0.088957,0.526201,1.580676,0.494443,0.982242,0.075964,Train,$> 0$,$d > 5$,2
division_1_size_5,0.406937,1.0,0.028525,0.477909,1.435462,0.396989,0.982472,0.035765,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,0.618445,1.0,0.079233,0.535789,1.697678,0.581733,0.981591,0.067710,Train,$< 3$,$2 < d \leq 5$,1


## FTIR-features (1900-800 cm$^{-1}$)

In [6]:
dataset_1900_800 = pd.read_csv(path_to_ftir_data_1900_800, sep=';', index_col=0)

print(dataset_1900_800.shape)
dataset_1900_800.head(3)

(75, 30)


,$I_{873}$,$I_{1105}$,$I_{1241}$,$I_{1394}$,$I_{1615}$,$I_{1726}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{873 \pm 25}$,$\overline{I}_{873 \pm 50}$,$\overline{I}_{1105 \pm 25}$,$\overline{I}_{1105 \pm 50}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1241 \pm 50}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1394 \pm 50}$,$\overline{I}_{1615 \pm 25}$,$\overline{I}_{1615 \pm 50}$,$\overline{I}_{1726 \pm 25}$,$\overline{I}_{1726 \pm 50}$,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,0.048208,0.681294,0.455066,0.760582,1.0,0.421995,0.424162,3.367146,0.426619,1.421995,0.571735,0.760582,0.363338,1.184569,0.047543,0.047916,0.628121,0.615327,0.447733,0.442677,0.738860,0.689531,0.944739,0.845640,0.415802,0.413416,Train,$> 0$,$d > 5$,2
division_1_size_5,0.091875,0.606138,0.427988,0.687458,1.0,0.383362,0.378730,3.196821,0.378004,1.383362,0.502368,0.687458,0.329999,1.126001,0.093349,0.096086,0.593910,0.562713,0.424918,0.421892,0.667947,0.619315,0.937481,0.831352,0.376555,0.371393,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,0.110321,0.836176,0.400950,0.696553,1.0,0.366658,0.436266,3.410658,0.429362,1.366658,0.523513,0.696553,0.406988,1.347446,0.113396,0.114054,0.816077,0.774107,0.401435,0.409080,0.678774,0.631706,0.957114,0.877520,0.366774,0.375602,Train,$< 3$,$2 < d \leq 5$,1


# Feature engineering

## Data wrangling

In [7]:
features_3800_2500 = dataset_3800_2500.loc[:, ~dataset_3800_2500.columns.isin(['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class'])]

print(features_3800_2500.shape)
features_3800_2500.head(3)

(75, 8)


,$I_{2928}$,$I_{3355}$,$I_{3725}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3355 \pm 50}$,$\overline{I}_{3725 \pm 50}$
division_1_size_bulk,0.491720,1.0,0.088957,0.526201,1.580676,0.494443,0.982242,0.075964
division_1_size_5,0.406937,1.0,0.028525,0.477909,1.435462,0.396989,0.982472,0.035765
division_1_size_3,0.618445,1.0,0.079233,0.535789,1.697678,0.581733,0.981591,0.067710


In [8]:
features_1900_800 = dataset_1900_800.loc[:, ~dataset_1900_800.columns.isin(['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class'])]

print(features_1900_800.shape)
features_1900_800.head(3)

(75, 26)


,$I_{873}$,$I_{1105}$,$I_{1241}$,$I_{1394}$,$I_{1615}$,$I_{1726}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{873 \pm 25}$,$\overline{I}_{873 \pm 50}$,$\overline{I}_{1105 \pm 25}$,$\overline{I}_{1105 \pm 50}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1241 \pm 50}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1394 \pm 50}$,$\overline{I}_{1615 \pm 25}$,$\overline{I}_{1615 \pm 50}$,$\overline{I}_{1726 \pm 25}$,$\overline{I}_{1726 \pm 50}$
division_1_size_bulk,0.048208,0.681294,0.455066,0.760582,1.0,0.421995,0.424162,3.367146,0.426619,1.421995,0.571735,0.760582,0.363338,1.184569,0.047543,0.047916,0.628121,0.615327,0.447733,0.442677,0.738860,0.689531,0.944739,0.845640,0.415802,0.413416
division_1_size_5,0.091875,0.606138,0.427988,0.687458,1.0,0.383362,0.378730,3.196821,0.378004,1.383362,0.502368,0.687458,0.329999,1.126001,0.093349,0.096086,0.593910,0.562713,0.424918,0.421892,0.667947,0.619315,0.937481,0.831352,0.376555,0.371393
division_1_size_3,0.110321,0.836176,0.400950,0.696553,1.0,0.366658,0.436266,3.410658,0.429362,1.366658,0.523513,0.696553,0.406988,1.347446,0.113396,0.114054,0.816077,0.774107,0.401435,0.409080,0.678774,0.631706,0.957114,0.877520,0.366774,0.375602


In [9]:
initial_features_df = pd.concat([features_3800_2500, features_1900_800], axis=1)

print(initial_features_df.shape)
initial_features_df.head(3)

(75, 34)


,$I_{2928}$,$I_{3355}$,$I_{3725}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3355 \pm 50}$,$\overline{I}_{3725 \pm 50}$,$I_{873}$,$I_{1105}$,$I_{1241}$,$I_{1394}$,$I_{1615}$,$I_{1726}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{873 \pm 25}$,$\overline{I}_{873 \pm 50}$,$\overline{I}_{1105 \pm 25}$,$\overline{I}_{1105 \pm 50}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1241 \pm 50}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1394 \pm 50}$,$\overline{I}_{1615 \pm 25}$,$\overline{I}_{1615 \pm 50}$,$\overline{I}_{1726 \pm 25}$,$\overline{I}_{1726 \pm 50}$
division_1_size_bulk,0.491720,1.0,0.088957,0.526201,1.580676,0.494443,0.982242,0.075964,0.048208,0.681294,0.455066,0.760582,1.0,0.421995,0.424162,3.367146,0.426619,1.421995,0.571735,0.760582,0.363338,1.184569,0.047543,0.047916,0.628121,0.615327,0.447733,0.442677,0.738860,0.689531,0.944739,0.845640,0.415802,0.413416
division_1_size_5,0.406937,1.0,0.028525,0.477909,1.435462,0.396989,0.982472,0.035765,0.091875,0.606138,0.427988,0.687458,1.0,0.383362,0.378730,3.196821,0.378004,1.383362,0.502368,0.687458,0.329999,1.126001,0.093349,0.096086,0.593910,0.562713,0.424918,0.421892,0.667947,0.619315,0.937481,0.831352,0.376555,0.371393
division_1_size_3,0.618445,1.0,0.079233,0.535789,1.697678,0.581733,0.981591,0.067710,0.110321,0.836176,0.400950,0.696553,1.0,0.366658,0.436266,3.410658,0.429362,1.366658,0.523513,0.696553,0.406988,1.347446,0.113396,0.114054,0.816077,0.774107,0.401435,0.409080,0.678774,0.631706,0.957114,0.877520,0.366774,0.375602


## Functions for processing

### Spectral region determination

In [10]:
def extract_center(name: str):
    nums = re.findall(r"\d+", name)
    return int(nums[0]) if nums else None

def get_interval_tag(center):
    if center is None:
        return None
    if 2500 <= center <= 3800:
        return "high"
    elif 800 <= center <= 1900:
        return "low"
    return None

### Features construction

In [11]:
def generate_transformed_features(data, feature_list, tolerance=50, zero_substitute=1e-8):
    data_transformed = pd.DataFrame(index=data.index)

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def wrap_log(col):
        return fr"$\log {clean(col)}$"

    def wrap_pow(col, power):
        return fr"$({clean(col)})^{{{power}}}$"

    def wrap_div(col1, col2):
        return fr"$\frac{{{clean(col1)}}}{{{clean(col2)}}}$"

    def extract_center(name):
        nums = re.findall(r'\d+', name)
        return int(nums[0]) if nums else None

    def get_interval_tag(center):
        if center is None:
            return None
        if 2500 <= center <= 3800:
            return "high"
        elif 800 <= center <= 1900:
            return "low"
        return None

    def extract_center_and_type(name):
        center = extract_center(name)
        is_avg = r'\overline{I}' in name or '±' in name
        return center, is_avg

    def is_near_duplicate(f1, f2):
        c1, avg1 = extract_center_and_type(f1)
        c2, avg2 = extract_center_and_type(f2)
        if c1 is None or c2 is None:
            return False
        return abs(c1 - c2) <= tolerance and (avg1 or avg2)

    def get_interval_kind(name: str):
        if r"\overline{I}" in name or "±" in name:
            return "average"
        if r"\Sigma I_{p," in name:
            return "sum"
        return None
    
        np.seterr(divide='ignore', invalid='ignore')

    for col in feature_list:
        base = data[col].copy()
        base_safe = base.replace(0, zero_substitute)

        data_transformed[wrap_log(col)] = np.log(base_safe)

        for power in [-1]:
            transformed = np.power(base_safe, power)
            transformed.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_transformed[wrap_pow(col, power)] = transformed

    for f1, f2 in combinations(feature_list, 2):
        if is_near_duplicate(f1, f2):
            continue
        
        kind1 = get_interval_kind(f1)
        kind2 = get_interval_kind(f2)
        if {kind1, kind2} == {"average", "sum"}:
            continue
        
        c1 = extract_center(f1)
        c2 = extract_center(f2)
        tag1 = get_interval_tag(c1)
        tag2 = get_interval_tag(c2)
        if tag1 != tag2:
            continue

        numerator = data[f1].replace(0, zero_substitute)
        denominator = data[f2].replace(0, zero_substitute)
        ratio = numerator / denominator
        ratio.replace([np.inf, -np.inf], np.nan, inplace=True)

        data_transformed[wrap_div(f1, f2)] = ratio

    return data_transformed


### Features classification

In [12]:
def classify_feature_type(base_columns, columns):
    base_columns = [col for col in base_columns if "Unnamed" not in col]
    columns = [col for col in columns if "Unnamed" not in col]

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def is_interval(col):
        col = clean(col)
        return r"\overline{I}_{" in col or r"\Sigma I_{p," in col

    def is_peak(col):
        return not is_interval(col)

    def extract_numbers(col):
        return list(map(int, re.findall(r"\d{3,4}", col)))

    def is_in_range(col, low, high):
        return any(low <= num <= high for num in extract_numbers(clean(col)))

    def classify(col, base=True):
        col_clean = clean(col)
        source = "intervals" if is_interval(col_clean) else "peaks"

        if is_in_range(col_clean, 2500, 3800):
            region = "3800_2500"
        elif is_in_range(col_clean, 800, 1900):
            region = "1900_800"
        else:
            region = "unknown"

        kind = "base" if base else "engineered"
        return f"{kind}_features_{source}_{region}"

    groups = {}

    for col in base_columns:
        key = classify(col, base=True)
        groups.setdefault(key, []).append(col)

    for col in columns:
        if col not in base_columns:
            key = classify(col, base=False)
            groups.setdefault(key, []).append(col)

    groups["base_features"] = base_columns
    groups["engineered_features"] = [col for col in columns if col not in base_columns]

    return groups

## Calculation process

In [13]:
all_features = initial_features_df.columns.tolist()
features_1900_800_list = [c for c in all_features if get_interval_tag(extract_center(c)) == "low"]
features_3800_2500_list = [c for c in all_features if get_interval_tag(extract_center(c)) == "high"]

print(
    f"Число фичей:",
    f"(*) 1900–800: {len(features_1900_800_list)}",
    f"(*) 3800–2500: {len(features_3800_2500_list)}",
    sep='\n'
)

Число фичей:
(*) 1900–800: 26
(*) 3800–2500: 8


### 3800—2500 cm$^{-1}$

In [14]:
engineered_3800_2500 = generate_transformed_features(
    data=features_3800_2500, 
    feature_list=features_3800_2500_list, 
    tolerance=0
)

In [15]:
nan_df = engineered_3800_2500.isna().sum()
nan_summary = nan_df[nan_df > 0].sort_values(ascending=False)

print(f"Столбцов с NaN-values: {nan_summary.shape[0]}")
display(nan_summary.head(10))

Столбцов с NaN-values: 0


Series([], dtype: int64)

In [16]:
all_features_3800_2500 = pd.merge(
    left=features_3800_2500,
    right=engineered_3800_2500,
    how='inner',
    left_index=True,
    right_index=True
)

print(all_features_3800_2500.shape)
all_features_3800_2500.head(3)

(75, 45)


,$I_{2928}$,$I_{3355}$,$I_{3725}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3355 \pm 50}$,$\overline{I}_{3725 \pm 50}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log I_{3355}$,$(I_{3355})^{-1}$,$\log I_{3725}$,$(I_{3725})^{-1}$,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,"$\log \Sigma I_{p,\ 3800\text{–}2500}$","$(\Sigma I_{p,\ 3800\text{–}2500})^{-1}$",$\log \overline{I}_{2928 \pm 50}$,$(\overline{I}_{2928 \pm 50})^{-1}$,$\log \overline{I}_{3355 \pm 50}$,$(\overline{I}_{3355 \pm 50})^{-1}$,$\log \overline{I}_{3725 \pm 50}$,$(\overline{I}_{3725 \pm 50})^{-1}$,$\frac{I_{2928}}{I_{3355}}$,$\frac{I_{2928}}{I_{3725}}$,$\frac{I_{2928}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{2928}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{2928}}{\overline{I}_{3355 \pm 50}}$,$\frac{I_{2928}}{\overline{I}_{3725 \pm 50}}$,$\frac{I_{3355}}{I_{3725}}$,$\frac{I_{3355}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3355}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3355}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3355}}{\overline{I}_{3725 \pm 50}}$,$\frac{I_{3725}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3725}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3725}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3725}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{2928 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3725 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3725 \pm 50}}$,$\frac{\overline{I}_{3355 \pm 50}}{\overline{I}_{3725 \pm 50}}$
division_1_size_bulk,0.491720,1.0,0.088957,0.526201,1.580676,0.494443,0.982242,0.075964,-0.709847,2.033679,0.0,1.0,-2.419606,11.241429,-0.642072,1.900414,0.457853,0.632641,-0.704324,2.022478,-0.017917,1.018079,-2.577498,13.164155,0.491720,5.527631,0.934471,0.311082,0.500609,6.473072,11.241429,1.900414,0.632641,2.022478,13.164155,0.169054,0.056278,0.179913,0.090565,1.064230,0.535714,6.926993,0.503382,6.508923,12.930390
division_1_size_5,0.406937,1.0,0.028525,0.477909,1.435462,0.396989,0.982472,0.035765,-0.899098,2.457385,0.0,1.0,-3.556968,35.056730,-0.738335,2.092448,0.361487,0.696640,-0.923846,2.518960,-0.017684,1.017841,-3.330798,27.960634,0.406937,14.265868,0.851494,0.283488,0.414197,11.378206,35.056730,2.092448,0.696640,2.518960,27.960634,0.059687,0.019872,0.071854,0.029034,1.203834,0.486436,13.362643,0.404072,11.100072,27.470532
division_1_size_3,0.618445,1.0,0.079233,0.535789,1.697678,0.581733,0.981591,0.067710,-0.480547,1.616959,0.0,1.0,-2.535360,12.620970,-0.624015,1.866407,0.529262,0.589040,-0.541743,1.719000,-0.018581,1.018755,-2.692524,14.768908,0.618445,7.805375,1.154270,0.364289,0.630044,9.133756,12.620970,1.866407,0.589040,1.719000,14.768908,0.147881,0.046672,0.136202,0.080719,0.921021,0.545837,7.913018,0.592644,8.591567,14.497024


In [17]:
base_columns = features_3800_2500.columns.tolist()
columns = all_features_3800_2500.columns.tolist()
features_description_dict_3800_2500 = classify_feature_type(base_columns=base_columns, columns=columns)

for feature_type, feature_lst in features_description_dict_3800_2500.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_peaks_3800_2500: 3
(*) base_features_intervals_3800_2500: 5
(*) engineered_features_peaks_3800_2500: 9
(*) engineered_features_intervals_3800_2500: 28
(*) base_features: 8
(*) engineered_features: 37


### 1900—800 cm$^{-1}$

In [18]:
engineered_1900_800 = generate_transformed_features(
    data=features_1900_800, 
    feature_list=features_1900_800_list, 
    tolerance=0
)

In [19]:
nan_df = engineered_1900_800.isna().sum()
nan_summary = nan_df[nan_df > 0].sort_values(ascending=False)

print(f"Столбцов с NaN-values: {nan_summary.shape[0]}")
display(nan_summary.head(10))

Столбцов с NaN-values: 0


Series([], dtype: int64)

In [20]:
all_features_1900_800 = pd.merge(
    left=features_1900_800,
    right=engineered_1900_800,
    how='inner',
    left_index=True,
    right_index=True
)

print(all_features_1900_800.shape)
all_features_1900_800.head(3)

(75, 320)


,$I_{873}$,$I_{1105}$,$I_{1241}$,$I_{1394}$,$I_{1615}$,$I_{1726}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{873 \pm 25}$,$\overline{I}_{873 \pm 50}$,$\overline{I}_{1105 \pm 25}$,$\overline{I}_{1105 \pm 50}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1241 \pm 50}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1394 \pm 50}$,$\overline{I}_{1615 \pm 25}$,$\overline{I}_{1615 \pm 50}$,$\overline{I}_{1726 \pm 25}$,$\overline{I}_{1726 \pm 50}$,$\log I_{873}$,$(I_{873})^{-1}$,$\log I_{1105}$,$(I_{1105})^{-1}$,$\log I_{1241}$,$(I_{1241})^{-1}$,$\log I_{1394}$,$(I_{1394})^{-1}$,$\log I_{1615}$,$(I_{1615})^{-1}$,$\log I_{1726}$,$(I_{1726})^{-1}$,$\log \overline{I}_{1900\text{–}800}$,$(\overline{I}_{1900\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}800}$","$(\Sigma I_{p,\ 1900\text{–}800})^{-1}$",$\log \overline{I}_{1900\text{–}1500}$,$(\overline{I}_{1900\text{–}1500})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}1500}$","$(\Sigma I_{p,\ 1900\text{–}1500})^{-1}$",$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,"$\log \Sigma I_{p,\ 1500\text{–}1300}$","$(\Sigma I_{p,\ 1500\text{–}1300})^{-1}$",$\log \overline{I}_{1300\text{–}800}$,$(\overline{I}_{1300\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1300\text{–}800}$","$(\Sigma I_{p,\ 1300\text{–}800})^{-1}$",$\log \overline{I}_{873 \pm 25}$,$(\overline{I}_{873 \pm 25})^{-1}$,$\log \overline{I}_{873 \pm 50}$,$(\overline{I}_{873 \pm 50})^{-1}$,$\log \overline{I}_{1105 \pm 25}$,$(\overline{I}_{1105 \pm 25})^{-1}$,$\log \overline{I}_{1105 \pm 50}$,$(\overline{I}_{1105 \pm 50})^{-1}$,$\log \overline{I}_{1241 \pm 25}$,$(\overline{I}_{1241 \pm 25})^{-1}$,$\log \overline{I}_{1241 \pm 50}$,$(\overline{I}_{1241 \pm 50})^{-1}$,$\log \overline{I}_{1394 \pm 25}$,$(\overline{I}_{1394 \pm 25})^{-1}$,$\log \overline{I}_{1394 \pm 50}$,$(\overline{I}_{1394 \pm 50})^{-1}$,$\log \overline{I}_{1615 \pm 25}$,$(\overline{I}_{1615 \pm 25})^{-1}$,$\log \overline{I}_{1615 \pm 50}$,$(\overline{I}_{1615 \pm 50})^{-1}$,$\log \overline{I}_{1726 \pm 25}$,$(\overline{I}_{1726 \pm 25})^{-1}$,$\log \overline{I}_{1726 \pm 50}$,$(\overline{I}_{1726 \pm 50})^{-1}$,$\frac{I_{873}}{I_{1105}}$,$\frac{I_{873}}{I_{1241}}$,$\frac{I_{873}}{I_{1394}}$,$\frac{I_{873}}{I_{1615}}$,$\frac{I_{873}}{I_{1726}}$,$\frac{I_{873}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{873}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{873}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{873}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{873}}{\overline{I}_{1105 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1105 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1241 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1394 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1615 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1615 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1726 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1726 \pm 50}}$,$\frac{I_{1105}}{I_{1241}}$,$\frac{I_{1105}}{I_{1394}}$,$\frac{I_{1105}}{I_{1615}}$,$\frac{I_{1105}}{I_{1726}}$,$\frac{I_{1105}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1105}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1105}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{1105}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{1105}}{\overline{I}_{873 \pm 25}}$,$\frac{I_{1105}}{\overline{I}_{873 \pm 50}}$,$\frac{I_{1105}}{\overl

In [21]:
base_columns = features_1900_800.columns.tolist()
columns = all_features_1900_800.columns.tolist()
features_description_dict_1900_800 = classify_feature_type(base_columns=base_columns, columns=columns)

for feature_type, feature_lst in features_description_dict_1900_800.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_peaks_1900_800: 6
(*) base_features_intervals_1900_800: 20
(*) engineered_features_peaks_1900_800: 27
(*) engineered_features_intervals_1900_800: 267
(*) base_features: 26
(*) engineered_features: 294


## Construction of datasets

### 3800—2500 cm$^{-1}$

In [22]:
dataset_3800_2500_part = dataset_3800_2500[['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class']]

print(dataset_3800_2500_part.shape)
dataset_3800_2500_part.head(3)

(75, 4)


,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,Train,$> 0$,$d > 5$,2
division_1_size_5,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,Train,$< 3$,$2 < d \leq 5$,1


In [23]:
dataset_3800_2500 = pd.merge(all_features_3800_2500, dataset_3800_2500_part, left_index=True, right_index=True, how='right')

print(dataset_3800_2500.shape)
display(dataset_3800_2500.head(3))
display(pd.DataFrame(dataset_3800_2500.isna().sum()).T)

(75, 49)


,$I_{2928}$,$I_{3355}$,$I_{3725}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3355 \pm 50}$,$\overline{I}_{3725 \pm 50}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log I_{3355}$,$(I_{3355})^{-1}$,$\log I_{3725}$,$(I_{3725})^{-1}$,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,"$\log \Sigma I_{p,\ 3800\text{–}2500}$","$(\Sigma I_{p,\ 3800\text{–}2500})^{-1}$",$\log \overline{I}_{2928 \pm 50}$,$(\overline{I}_{2928 \pm 50})^{-1}$,$\log \overline{I}_{3355 \pm 50}$,$(\overline{I}_{3355 \pm 50})^{-1}$,$\log \overline{I}_{3725 \pm 50}$,$(\overline{I}_{3725 \pm 50})^{-1}$,$\frac{I_{2928}}{I_{3355}}$,$\frac{I_{2928}}{I_{3725}}$,$\frac{I_{2928}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{2928}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{2928}}{\overline{I}_{3355 \pm 50}}$,$\frac{I_{2928}}{\overline{I}_{3725 \pm 50}}$,$\frac{I_{3355}}{I_{3725}}$,$\frac{I_{3355}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3355}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3355}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3355}}{\overline{I}_{3725 \pm 50}}$,$\frac{I_{3725}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3725}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3725}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3725}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{2928 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3725 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3725 \pm 50}}$,$\frac{\overline{I}_{3355 \pm 50}}{\overline{I}_{3725 \pm 50}}$,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,0.491720,1.0,0.088957,0.526201,1.580676,0.494443,0.982242,0.075964,-0.709847,2.033679,0.0,1.0,-2.419606,11.241429,-0.642072,1.900414,0.457853,0.632641,-0.704324,2.022478,-0.017917,1.018079,-2.577498,13.164155,0.491720,5.527631,0.934471,0.311082,0.500609,6.473072,11.241429,1.900414,0.632641,2.022478,13.164155,0.169054,0.056278,0.179913,0.090565,1.064230,0.535714,6.926993,0.503382,6.508923,12.930390,Train,$> 0$,$d > 5$,2
division_1_size_5,0.406937,1.0,0.028525,0.477909,1.435462,0.396989,0.982472,0.035765,-0.899098,2.457385,0.0,1.0,-3.556968,35.056730,-0.738335,2.092448,0.361487,0.696640,-0.923846,2.518960,-0.017684,1.017841,-3.330798,27.960634,0.406937,14.265868,0.851494,0.283488,0.414197,11.378206,35.056730,2.092448,0.696640,2.518960,27.960634,0.059687,0.019872,0.071854,0.029034,1.203834,0.486436,13.362643,0.404072,11.100072,27.470532,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,0.618445,1.0,0.079233,0.535789,1.697678,0.581733,0.981591,0.067710,-0.480547,1.616959,0.0,1.0,-2.535360,12.620970,-0.624015,1.866407,0.529262,0.589040,-0.541743,1.719000,-0.018581,1.018755,-2.692524,14.768908,0.618445,7.805375,1.154270,0.364289,0.630044,9.133756,12.620970,1.866407,0.589040,1.719000,14.768908,0.147881,0.046672,0.136202,0.080719,0.921021,0.545837,7.913018,0.592644,8.591567,14.497024,Train,$< 3$,$2 < d \leq 5$,1


,$I_{2928}$,$I_{3355}$,$I_{3725}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3355 \pm 50}$,$\overline{I}_{3725 \pm 50}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log I_{3355}$,$(I_{3355})^{-1}$,$\log I_{3725}$,$(I_{3725})^{-1}$,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,"$\log \Sigma I_{p,\ 3800\text{–}2500}$","$(\Sigma I_{p,\ 3800\text{–}2500})^{-1}$",$\log \overline{I}_{2928 \pm 50}$,$(\overline{I}_{2928 \pm 50})^{-1}$,$\log \overline{I}_{3355 \pm 50}$,$(\overline{I}_{3355 \pm 50})^{-1}$,$\log \overline{I}_{3725 \pm 50}$,$(\overline{I}_{3725 \pm 50})^{-1}$,$\frac{I_{2928}}{I_{3355}}$,$\frac{I_{2928}}{I_{3725}}$,$\frac{I_{2928}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{2928}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{2928}}{\overline{I}_{3355 \pm 50}}$,$\frac{I_{2928}}{\overline{I}_{3725 \pm 50}}$,$\frac{I_{3355}}{I_{3725}}$,$\frac{I_{3355}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3355}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3355}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3355}}{\overline{I}_{3725 \pm 50}}$,$\frac{I_{3725}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3725}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3725}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3725}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{2928 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3725 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3355 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3725 \pm 50}}$,$\frac{\overline{I}_{3355 \pm 50}}{\overline{I}_{3725 \pm 50}}$,Split,Fraction_hue,Fraction_grouped_hue,Class
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 1900—800 cm$^{-1}$

In [24]:
dataset_1900_800_part = dataset_1900_800[['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class']]

print(dataset_1900_800_part.shape)
dataset_1900_800_part.head(3)

(75, 4)


,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,Train,$> 0$,$d > 5$,2
division_1_size_5,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,Train,$< 3$,$2 < d \leq 5$,1


In [25]:
dataset_1900_800= pd.merge(all_features_1900_800, dataset_1900_800_part, left_index=True, right_index=True, how='right')

print(dataset_1900_800.shape)
display(dataset_1900_800.head(3))
display(pd.DataFrame(dataset_1900_800.isna().sum()).T)

(75, 324)


,$I_{873}$,$I_{1105}$,$I_{1241}$,$I_{1394}$,$I_{1615}$,$I_{1726}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{873 \pm 25}$,$\overline{I}_{873 \pm 50}$,$\overline{I}_{1105 \pm 25}$,$\overline{I}_{1105 \pm 50}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1241 \pm 50}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1394 \pm 50}$,$\overline{I}_{1615 \pm 25}$,$\overline{I}_{1615 \pm 50}$,$\overline{I}_{1726 \pm 25}$,$\overline{I}_{1726 \pm 50}$,$\log I_{873}$,$(I_{873})^{-1}$,$\log I_{1105}$,$(I_{1105})^{-1}$,$\log I_{1241}$,$(I_{1241})^{-1}$,$\log I_{1394}$,$(I_{1394})^{-1}$,$\log I_{1615}$,$(I_{1615})^{-1}$,$\log I_{1726}$,$(I_{1726})^{-1}$,$\log \overline{I}_{1900\text{–}800}$,$(\overline{I}_{1900\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}800}$","$(\Sigma I_{p,\ 1900\text{–}800})^{-1}$",$\log \overline{I}_{1900\text{–}1500}$,$(\overline{I}_{1900\text{–}1500})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}1500}$","$(\Sigma I_{p,\ 1900\text{–}1500})^{-1}$",$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,"$\log \Sigma I_{p,\ 1500\text{–}1300}$","$(\Sigma I_{p,\ 1500\text{–}1300})^{-1}$",$\log \overline{I}_{1300\text{–}800}$,$(\overline{I}_{1300\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1300\text{–}800}$","$(\Sigma I_{p,\ 1300\text{–}800})^{-1}$",$\log \overline{I}_{873 \pm 25}$,$(\overline{I}_{873 \pm 25})^{-1}$,$\log \overline{I}_{873 \pm 50}$,$(\overline{I}_{873 \pm 50})^{-1}$,$\log \overline{I}_{1105 \pm 25}$,$(\overline{I}_{1105 \pm 25})^{-1}$,$\log \overline{I}_{1105 \pm 50}$,$(\overline{I}_{1105 \pm 50})^{-1}$,$\log \overline{I}_{1241 \pm 25}$,$(\overline{I}_{1241 \pm 25})^{-1}$,$\log \overline{I}_{1241 \pm 50}$,$(\overline{I}_{1241 \pm 50})^{-1}$,$\log \overline{I}_{1394 \pm 25}$,$(\overline{I}_{1394 \pm 25})^{-1}$,$\log \overline{I}_{1394 \pm 50}$,$(\overline{I}_{1394 \pm 50})^{-1}$,$\log \overline{I}_{1615 \pm 25}$,$(\overline{I}_{1615 \pm 25})^{-1}$,$\log \overline{I}_{1615 \pm 50}$,$(\overline{I}_{1615 \pm 50})^{-1}$,$\log \overline{I}_{1726 \pm 25}$,$(\overline{I}_{1726 \pm 25})^{-1}$,$\log \overline{I}_{1726 \pm 50}$,$(\overline{I}_{1726 \pm 50})^{-1}$,$\frac{I_{873}}{I_{1105}}$,$\frac{I_{873}}{I_{1241}}$,$\frac{I_{873}}{I_{1394}}$,$\frac{I_{873}}{I_{1615}}$,$\frac{I_{873}}{I_{1726}}$,$\frac{I_{873}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{873}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{873}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{873}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{873}}{\overline{I}_{1105 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1105 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1241 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1394 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1615 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1615 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1726 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1726 \pm 50}}$,$\frac{I_{1105}}{I_{1241}}$,$\frac{I_{1105}}{I_{1394}}$,$\frac{I_{1105}}{I_{1615}}$,$\frac{I_{1105}}{I_{1726}}$,$\frac{I_{1105}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1105}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1105}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{1105}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{1105}}{\overline{I}_{873 \pm 25}}$,$\frac{I_{1105}}{\overline{I}_{873 \pm 50}}$,$\frac{I_{1105}}{\overl

,$I_{873}$,$I_{1105}$,$I_{1241}$,$I_{1394}$,$I_{1615}$,$I_{1726}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{873 \pm 25}$,$\overline{I}_{873 \pm 50}$,$\overline{I}_{1105 \pm 25}$,$\overline{I}_{1105 \pm 50}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1241 \pm 50}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1394 \pm 50}$,$\overline{I}_{1615 \pm 25}$,$\overline{I}_{1615 \pm 50}$,$\overline{I}_{1726 \pm 25}$,$\overline{I}_{1726 \pm 50}$,$\log I_{873}$,$(I_{873})^{-1}$,$\log I_{1105}$,$(I_{1105})^{-1}$,$\log I_{1241}$,$(I_{1241})^{-1}$,$\log I_{1394}$,$(I_{1394})^{-1}$,$\log I_{1615}$,$(I_{1615})^{-1}$,$\log I_{1726}$,$(I_{1726})^{-1}$,$\log \overline{I}_{1900\text{–}800}$,$(\overline{I}_{1900\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}800}$","$(\Sigma I_{p,\ 1900\text{–}800})^{-1}$",$\log \overline{I}_{1900\text{–}1500}$,$(\overline{I}_{1900\text{–}1500})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}1500}$","$(\Sigma I_{p,\ 1900\text{–}1500})^{-1}$",$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,"$\log \Sigma I_{p,\ 1500\text{–}1300}$","$(\Sigma I_{p,\ 1500\text{–}1300})^{-1}$",$\log \overline{I}_{1300\text{–}800}$,$(\overline{I}_{1300\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1300\text{–}800}$","$(\Sigma I_{p,\ 1300\text{–}800})^{-1}$",$\log \overline{I}_{873 \pm 25}$,$(\overline{I}_{873 \pm 25})^{-1}$,$\log \overline{I}_{873 \pm 50}$,$(\overline{I}_{873 \pm 50})^{-1}$,$\log \overline{I}_{1105 \pm 25}$,$(\overline{I}_{1105 \pm 25})^{-1}$,$\log \overline{I}_{1105 \pm 50}$,$(\overline{I}_{1105 \pm 50})^{-1}$,$\log \overline{I}_{1241 \pm 25}$,$(\overline{I}_{1241 \pm 25})^{-1}$,$\log \overline{I}_{1241 \pm 50}$,$(\overline{I}_{1241 \pm 50})^{-1}$,$\log \overline{I}_{1394 \pm 25}$,$(\overline{I}_{1394 \pm 25})^{-1}$,$\log \overline{I}_{1394 \pm 50}$,$(\overline{I}_{1394 \pm 50})^{-1}$,$\log \overline{I}_{1615 \pm 25}$,$(\overline{I}_{1615 \pm 25})^{-1}$,$\log \overline{I}_{1615 \pm 50}$,$(\overline{I}_{1615 \pm 50})^{-1}$,$\log \overline{I}_{1726 \pm 25}$,$(\overline{I}_{1726 \pm 25})^{-1}$,$\log \overline{I}_{1726 \pm 50}$,$(\overline{I}_{1726 \pm 50})^{-1}$,$\frac{I_{873}}{I_{1105}}$,$\frac{I_{873}}{I_{1241}}$,$\frac{I_{873}}{I_{1394}}$,$\frac{I_{873}}{I_{1615}}$,$\frac{I_{873}}{I_{1726}}$,$\frac{I_{873}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{873}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{873}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{873}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{873}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{873}}{\overline{I}_{1105 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1105 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1241 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1394 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1615 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1615 \pm 50}}$,$\frac{I_{873}}{\overline{I}_{1726 \pm 25}}$,$\frac{I_{873}}{\overline{I}_{1726 \pm 50}}$,$\frac{I_{1105}}{I_{1241}}$,$\frac{I_{1105}}{I_{1394}}$,$\frac{I_{1105}}{I_{1615}}$,$\frac{I_{1105}}{I_{1726}}$,$\frac{I_{1105}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1105}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1105}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{1105}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{1105}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{1105}}{\overline{I}_{873 \pm 25}}$,$\frac{I_{1105}}{\overline{I}_{873 \pm 50}}$,$\frac{I_{1105}}{\overl

# Saving data

In [26]:
dataset_3800_2500.to_csv('./../data/processed_data/dataset_3800_2500_engineered.csv', sep=';')
dataset_1900_800.to_csv('./../data/processed_data/dataset_1900_800_engineered.csv', sep=';')

In [27]:
with open('./../data/processed_data/features_description_3800_2500.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict_3800_2500, f, ensure_ascii=False, indent=2)

In [28]:
with open('./../data/processed_data/features_description_1900_800.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict_1900_800, f, ensure_ascii=False, indent=2)